In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from keras import regularizers
from glob import glob
import librosa.display
import librosa

In [11]:
# import competition dataset
x_train = np.load('train_data.npy')
y_train = pd.read_csv('train_labels.csv')
y_train = np.array(y_train)
y_train = y_train[:,1]

x_test = np.load('test_data.npy')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# reshape
x_train = x_train.reshape(-1,210,210,1)
x_test = x_test.reshape(-1,210,210,1)

#print(x_train)
print(x_train.shape)

y_train = np_utils.to_categorical(y_train,2)

(3999, 210, 210, 1)


In [ ]:
# #Normalization 

# x_min = x_train.min(axis=(1, 2), keepdims=True)
# x_max = x_train.max(axis=(1, 2), keepdims=True)
# x_train = (x_train - x_min)/(x_max-x_min)
# print(x_train.shape)


In [31]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5,5),
                 activation='relu',
                 kernel_regularizer=regularizers.l1(0.001),
                 input_shape=(210,210,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=5))

model.add(Conv2D(64, (5, 5), activation='relu',
          kernel_regularizer=regularizers.l1(0.001)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=7))

model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=5))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 206, 206, 32)      832       
_________________________________________________________________
batch_normalization_19 (Batc (None, 206, 206, 32)      128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 41, 41, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 37, 37, 64)        51264     
_________________________________________________________________
batch_normalization_20 (Batc (None, 37, 37, 64)        256       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 33, 33, 128)       204928    
_________________________________________________________________
batch_normalization_21 (Batc (None, 33, 33, 128)       512       
__________

In [33]:
# compile model
adam = optimizers.Adam(lr=.0001) 
model.compile(loss='binary_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

In [34]:
print(x_train.shape,y_train.shape)

(3999, 210, 210, 1) (3999, 2)


In [35]:
from keras.callbacks import EarlyStopping
EarlyStopping = EarlyStopping(monitor='val_loss',patience=5,verbose=0,mode='auto')
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_split=0.1,callbacks=[EarlyStopping])

Train on 3599 samples, validate on 400 samples
Epoch 1/30
3599/3599 [==============================] - 173s 48ms/step - loss: 1.6788 - acc: 0.8979 - val_loss: 1.4517 - val_acc: 0.9213
Epoch 2/30
3599/3599 [==============================] - 164s 46ms/step - loss: 1.4490 - acc: 0.9311 - val_loss: 1.3692 - val_acc: 0.9550
Epoch 3/30
3599/3599 [==============================] - 166s 46ms/step - loss: 1.3753 - acc: 0.9417 - val_loss: 1.3241 - val_acc: 0.9475
Epoch 4/30
3599/3599 [==============================] - 179s 50ms/step - loss: 1.3076 - acc: 0.9508 - val_loss: 1.2790 - val_acc: 0.9500
Epoch 5/30
3599/3599 [==============================] - 180s 50ms/step - loss: 1.2530 - acc: 0.9503 - val_loss: 1.2140 - val_acc: 0.9625
Epoch 6/30
3599/3599 [==============================] - 176s 49ms/step - loss: 1.1992 - acc: 0.9537 - val_loss: 1.1588 - val_acc: 0.9600
Epoch 7/30
3599/3599 [==============================] - 166s 46ms/step - loss: 1.1348 - acc: 0.9615 - val_loss: 1.0937 - val_acc: 0

In [ ]:
# make prediction
y_predict = model.predict_classes(x_test)
print(y_predict[:500])

In [ ]:
# save the prediction to csv
prediction = pd.DataFrame(y_predict,columns=['label']).to_csv('submission_2dCNN')

In [ ]:
model.save("2dCNN_savemodel.h5")
print("Saved model to disk")

In [ ]:
#// loss: 0.0456 - acc: 0.9829 - val_loss: 0.0777 - val_acc: 0.9738
#//loss: 0.0399 - acc: 0.9867 - val_loss: 0.1112 - val_acc: 0.9625
# add l1 //loss: 0.3446 - acc: 0.9883 - val_loss: 0.4493 - val_acc: 0.9513